## import Python packages

In [2]:
from lyricsgenius import Genius
import pandas as pd
import requests
from itertools import chain
from collections import Counter
from string import punctuation
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
import re
tbl = str.maketrans({ord(ch):" " for ch in punctuation})

## authenticate Genius API

In [2]:
token  = 'zvYH5XVlo2URyfxjtHK-4aypOPl63Dved-diSY-FK82a6hAOgq7yNUIsrG1dchEC'
genius = Genius(token)

## authenticate Spotify API

In [3]:
cid  ='4bbdc8c4be9f415080b262deb3e33c16'
secret ='8b4a63460af64e30b702d27c1dafda4f'

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

### Below takes in all the different LVMH brands

In [5]:
lvmh = ['birkin','henny','hennessy','dior','louis','lv','hermes','moet','louis vuitton','vuitton','dom pérignon',
       'dom','perignon','hublot','givenchy','fendi','fenty','tiffany','christian','christian dior','marc',
        'bulgari','marc jacob','jacob']

### Below function pulls the lyrics of each song and counts the frequency of how many times a LVMH brand is 
### mentioned in the song's lyrics

In [7]:
def lyric_counter(song_df,word_list):
    total_counter = {}
    lyrics = song_df['lyrics']
    artist = song_df['artist']
    for i in range(len(lyrics)):
        print(album_df['song'][i])
        lyric = lyrics[i]
        count = Counter(lyric.lower().translate(tbl).split())
        d =  {key.lower(): count[key] for key in count if key in word_list}
        total_counter = dict(Counter(d)+Counter(total_counter))
    return total_counter
    

### Below function takes a Spotify playlist and a given year, pulls the song, artist and song ID from Spotify. 
### Using the song and artist, it uses Genius API to search for the song and its corresponding meta data. Both the
### Spotify and Genius API is saved in the dataframe and then output to a .csv file 

In [ ]:
def get_songs(playlist,year):
    song_row = pd.DataFrame()
    playlist = sp.playlist(playlist)
    song_df = pd.DataFrame()
    for track in range(len(playlist['tracks']['items'])):
        try:
            print(track)
            song_name = playlist['tracks']['items'][track]['track']['name']
            sp_artist = playlist['tracks']['items'][track]['track']['artists'][0]['name']
            sp_song_id = playlist['tracks']['items'][track]['track']['id']
            sp_artist_id = playlist['tracks']['items'][track]['track']['artists'][0]['id']
            song_name = re.sub(r"\([^()]*\)", "", song_name)
            song_search = genius.search_song(song_name, sp_artist)

            song_json = song_search.to_json()
            song_dict = json.loads(song_json)
            time.sleep(1)
            song_id = song_dict['id']
            df_artist = sp_artist
            song = song_name
            lyrics = genius.lyrics(song_id)
            song_id = str(song_dict['id'])
            song_df = song_df.append(
            {'rtist': song_search.artist,
             'year':year,
            'song' : song_name,
            'Genius artist_id':str(song_dict['primary_artist']['id']),
            'Genius song_id': song_id,
             'Spotify song_id': sp_song_id,
             'Spotify artist_id': sp_artist_id,
            'lyrics': lyrics},
            ignore_index=True
            )
        except:
            pass
        
    file_name = year+'rap_lyrics_database.csv'
    song_df.to_csv(file_name)
    print(f'Completed storing all lyrics for {year}')
    return song_df    

### Creates a Pandas dataframe for the counts of LVMH brands in lyrics

In [9]:
def create_counts(df,year):
    counts = lyric_counter(df,lvmh)
    counts_df = pd.DataFrame.from_dict(counts,orient='index').rename(columns ={0:year})
    return counts_df

### Python dictionary with all of years and Spotify URIs of Spotify Playlist of top Rap songs for that respective
### year

In [ ]:
playlists = {'2016':'6hAQqJYxe5mt9jWQjjwZde',
             '2017':'5ZXRYuiSQZeQKRhDFSgO3b',
             '2018':'5FyRUTsjmvQqBZnnnjFUmq',
             '2019':'1BCCebMPU7qCgEg8oy0cq4',
             '2020':'6Pw0qVok3XrfyER6ZvFD5V',
}

### Loop through each year & playlist, and count LVMH brands for each song, output the count dataframe to a .csv file

In [ ]:
for year, playlist in playlists.items():
    try:
        songs_all = get_songs(playlist,year)
        counts_all = create_counts(songs_all,year)
        count_filename = year+'lvmh_counts_database.csv'
        counts_all.to_csv(count_filename)
        print(f'Completed all counts for {year}')
    except TypeError:
        print(f'Error found in {year}, skipping to next')
        time.sleep(5)

### Read all the .csv files into Pandas dataframes

In [ ]:
counts_2020 = pd.read_csv('2020lvmh_counts_database.csv')
counts_2019 = pd.read_csv('2019lvmh_counts_database.csv')
counts_2018 = pd.read_csv('2018lvmh_counts_database.csv')
counts_2017 = pd.read_csv('2017lvmh_counts_database.csv')
counts_2016 = pd.read_csv('2016lvmh_counts_database.csv')
counts_2015 = pd.read_csv('2015lvmh_counts_database.csv')


### Merge all of them to one big dataframe

In [ ]:
counts = counts_2020.merge(counts_2019,how='outer')
counts = counts.merge(counts_2018,how='outer')
counts = counts.merge(counts_2017,how='outer')
counts = counts.merge(counts_2016,how='outer')
counts = counts.merge(counts_2015,how='outer')

### Rename columns into years and brands

In [ ]:
counts.rename(columns={'Unnamed: 0':'Brands'},inplace=True)

In [ ]:
counts['Total']= counts.sum(axis=1,numeric_only=True)
counts = counts[['Brands','2015','2016','2017','2018','2019','2020','Total']]
counts = counts.sort_values(by='Total',ascending=False)
counts = counts.set_index('Brands')
counts

### Aggregate counts of any brands with multiple names(i.e henny and hennessy) into one

In [ ]:
hennessy = counts.loc['henny'] + counts.loc['hennessy']
louisv = counts.loc['louis'] + counts.loc['vuitton']
christiandior = counts.loc['christian'] + counts.loc['dior']
marcjacobs = counts.loc['marc'] + counts.loc['jacob']
hennessy.name = 'henny/hennessy'
louisv.name = 'louis vuitton'
christiandior.name = 'christian dior'
marcjacobs.name = 'marc jacobs'
counts = counts.append([hennessy])
counts = counts.append([louisv])
counts = counts.append([christiandior])
counts = counts.append([marcjacobs])

### Sort by total

In [ ]:
counts = counts.sort_values(by='Total',ascending=False)
counts

## Next step: Now that we have all rap songs, their artists, and the frequency of using LVMH brands, next step is
## to see which of these brands are being streamed the most using Spotify's Chart data from spotifychart.com

### Download all the Spotify streaming data and merge it into one big dataframe

In [ ]:
import os
path = '/Users/Arturo/Desktop/Cacicazgo/Code/Genius-API/spotify_streams'
file_list = os.listdir(path) 
final_df = pd.DataFrame()
for file in file_list:
    newpath = os.path.join(path, file)
    df = pd.read_csv(newpath)
    print(newpath)
    final_df = final_df.append(df)

### Rename columns 

In [ ]:
final_df= final_df.rename(columns = {'Unnamed: 0':'Position','Unnamed: 1':'Song Name','Unnamed: 2':'Artist','Note that these figures are generated using a formula that protects against any artificial inflation of chart positions.':'Streams',
                                     'Unnamed: 4':'URL'})
final_df = final_df[1:]
final_df = final_df[~final_df['Streams'].isin(['Streams'])]
final_df



### Aggregate all the streams since 2017 and group them by song, artist, and Spotify URI

In [ ]:
streams = final_df.groupby(['Song Name','Artist','URL']).agg(
    {
    'Streams':sum
    }
).sort_values(by='Streams',ascending=False)
streams.to_csv('spotify_streams_2017_to_02_26_2021.csv')

In [ ]:
### Merge the streams table with the lyrics table on the song name

In [12]:
final_stream_lyrics_table = streams_final_table.merge(lyrics_df,how='inner',left_on='Song Name',right_on='song')

In [13]:
final_stream_table = final_stream_lyrics_table.drop_duplicates(subset=['Song Name']).reset_index()
final_stream_table

,index,Song Name,Artist,URL,Streams,Genius artist_id,Genius song_id,Spotify artist_id,Spotify song_id,artist,lyrics,song
0,0,SAD!,XXXTENTACION,https://open.spotify.com/track/3ee8Jmje8o58CHK...,577901267.0,396565,3430186,15UsOTVnJzReFVN1VCnxy4,3ee8Jmje8o58CHK66QrVC2,XXXTENTACION,[Intro]\nYeah\n\n[Chorus]\nWho am I? Someone t...,SAD!
1,4,SICKO MODE,Travis Scott,https://open.spotify.com/track/2xLMifQCjDGFmkH...,550670975.0,20185,3876994,0Y5tJX1MQlPlqiwlOH1tJY,2xLMifQCjDGFmkHkpNLD9h,Travis Scott,"[Part I]\n\n[Intro: Drake]\nAstro, yeah\nSun i...",SICKO MODE
2,5,goosebumps,Travis Scott,https://open.spotify.com/track/6gBFPUFcJLzWGx4...,549870433.0,20185,2849767,0Y5tJX1MQlPlqiwlOH1tJY,6gBFPUFcJLzWGx4lenP6h2,Travis Scott,[Intro: Travis Scott]\nYeah\n7:30 in the night...,goosebumps
3,6,Jocelyn Flores,XXXTENTACION,https://open.spotify.com/track/7m9OqQk4RVRkw9J...,532330080.0,396565,3185661,15UsOTVnJzReFVN1VCnxy4,7m9OqQk4RVRkw9JJdeAw96,XXXTENTACION,"[Intro: Shiloh Dynasty]\nI know you so well, s...",Jocelyn Flores
4,7,HUMBLE.,Kendrick Lamar,https://open.spotify.com/track/7KXjTSCq5nL1LoY...,527892188.0,1421,3039923,2YZyLoL8N0Wb9xBt1NhZWg,7KXjTSCq5nL1LoYtL7XAwS,Kendrick Lamar,[Intro]\nNobody pray for me\nIt been that day ...,HUMBLE.
...,...,...,...,...,...,...,...,...,...,...,...,...
2839,5188,Middle,DJ Snake,https://open.spotify.com/track/4OZEu0Vwq0RB2LA...,1037663.0,41287,2331497,540vIaP2JwjQb9dm3aArA4,4OZEu0Vwq0RB2LAq14v99n,DJ Snake,[Verse 1]\nStarin' at two different views on y...,Middle
2840,5189,Wake Up Where You Are,State of Sound,https://open.spotify.com/track/2s76aKBWKShlwPC...,1029544.0,1005514,2876087,05o1zW8b6PV6rEquXr74sB,2s76aKBWKShlwPC4xTtEEE,State of Sound,[Verse 1]\nNever thought that I would love aga...,Wake Up Where You Are
2841,5190,Nothing Without You,The Weeknd,https://open.spotify.com/track/3Te3gG6MPEDNqLG...,1017468.0,2358,2911563,1Xyo4u8uXC1ZmMpatF05PJ,3Te3gG6MPEDNqLGzWrjKC3,The Weeknd,"[Verse 1]\nI realized, I belong to you\nI feel...",Nothing Without You
2842,5191,A Lonely Night,The Weeknd,https://open.spotify.com/track/4UknpXoLjlB7Mao...,994402.0,2358,2917435,1Xyo4u8uXC1ZmMpatF05PJ,4UknpXoLjlB7Mao4UWcJ2o,The Weeknd,"[Intro]\nGassed up, yeah, yeah\nHey, na-na-na-...",A Lonely Night


In [ ]:
## Rename all of the columns accordingly

In [32]:
final_stream_table = final_stream_table[['Song Name','Artist','URL','Streams','Genius artist_id','Genius song_id',
                                        'Spotify artist_id','Spotify song_id','artist','lyrics','song']]
final_stream_table

,Song Name,Artist,URL,Streams,Genius artist_id,Genius song_id,Spotify artist_id,Spotify song_id,artist,lyrics,song
0,SAD!,XXXTENTACION,https://open.spotify.com/track/3ee8Jmje8o58CHK...,577901267.0,396565,3430186,15UsOTVnJzReFVN1VCnxy4,3ee8Jmje8o58CHK66QrVC2,XXXTENTACION,[Intro]\nYeah\n\n[Chorus]\nWho am I? Someone t...,SAD!
1,SICKO MODE,Travis Scott,https://open.spotify.com/track/2xLMifQCjDGFmkH...,550670975.0,20185,3876994,0Y5tJX1MQlPlqiwlOH1tJY,2xLMifQCjDGFmkHkpNLD9h,Travis Scott,"[Part I]\n\n[Intro: Drake]\nAstro, yeah\nSun i...",SICKO MODE
2,goosebumps,Travis Scott,https://open.spotify.com/track/6gBFPUFcJLzWGx4...,549870433.0,20185,2849767,0Y5tJX1MQlPlqiwlOH1tJY,6gBFPUFcJLzWGx4lenP6h2,Travis Scott,[Intro: Travis Scott]\nYeah\n7:30 in the night...,goosebumps
3,Jocelyn Flores,XXXTENTACION,https://open.spotify.com/track/7m9OqQk4RVRkw9J...,532330080.0,396565,3185661,15UsOTVnJzReFVN1VCnxy4,7m9OqQk4RVRkw9JJdeAw96,XXXTENTACION,"[Intro: Shiloh Dynasty]\nI know you so well, s...",Jocelyn Flores
4,HUMBLE.,Kendrick Lamar,https://open.spotify.com/track/7KXjTSCq5nL1LoY...,527892188.0,1421,3039923,2YZyLoL8N0Wb9xBt1NhZWg,7KXjTSCq5nL1LoYtL7XAwS,Kendrick Lamar,[Intro]\nNobody pray for me\nIt been that day ...,HUMBLE.
...,...,...,...,...,...,...,...,...,...,...,...
2839,Middle,DJ Snake,https://open.spotify.com/track/4OZEu0Vwq0RB2LA...,1037663.0,41287,2331497,540vIaP2JwjQb9dm3aArA4,4OZEu0Vwq0RB2LAq14v99n,DJ Snake,[Verse 1]\nStarin' at two different views on y...,Middle
2840,Wake Up Where You Are,State of Sound,https://open.spotify.com/track/2s76aKBWKShlwPC...,1029544.0,1005514,2876087,05o1zW8b6PV6rEquXr74sB,2s76aKBWKShlwPC4xTtEEE,State of Sound,[Verse 1]\nNever thought that I would love aga...,Wake Up Where You Are
2841,Nothing Without You,The Weeknd,https://open.spotify.com/track/3Te3gG6MPEDNqLG...,1017468.0,2358,2911563,1Xyo4u8uXC1ZmMpatF05PJ,3Te3gG6MPEDNqLGzWrjKC3,The Weeknd,"[Verse 1]\nI realized, I belong to you\nI feel...",Nothing Without You
2842,A Lonely Night,The Weeknd,https://open.spotify.com/track/4UknpXoLjlB7Mao...,994402.0,2358,2917435,1Xyo4u8uXC1ZmMpatF05PJ,4UknpXoLjlB7Mao4UWcJ2o,The Weeknd,"[Intro]\nGassed up, yeah, yeah\nHey, na-na-na-...",A Lonely Night


In [ ]:
### Count which are the most common brands overall under the LVMH umbrella

In [30]:
streams_lvmh_counter = lyric_counter(final_stream_table,lvmh)
stream_counts = pd.DataFrame.from_dict(streams_lvmh_counter,orient='index').rename(columns ={0:'counts'})
stream_counts.sort_values(by='counts',ascending=False)

,counts
louis,164
dior,73
christian,59
henny,51
fendi,49
dom,48
hennessy,37
birkin,35
givenchy,27
jacob,27


In [ ]:
### Count brands for each song, taking into account their total streams

In [6]:
final_stream_df = pd.DataFrame()
for i in range(len(final_stream_table['lyrics'])):
    for brand in lvmh_new:
        
        song = final_stream_table['Song Name'][i]
        artist = final_stream_table['artist'][i]
        lyric = final_stream_table['lyrics'][i]
        streams = final_stream_table['Streams'][i]
        
        lyric_list = lyric.lower().translate(tbl).split()
        x = lyric_list.count(brand)
        
        if x > 0:
            final_stream_df = final_stream_df.append(
                {'Song':song,
                 'Artist':artist,
                 'Brand':brand,
                 'Brand Count':x,
                 'Streams':streams,
                },
                ignore_index=True
            )
            print(f'{song} by {artist} has the brand {brand} {x} times.')

### Final dataframe with artist, brand, song, streams, and brand count

In [38]:
final_stream_df

,Artist,Brand,Brand Count,Song,Streams
0,Travis Scott,louis,1.0,SICKO MODE,550670975.0
1,Post Malone,henny,1.0,Better Now,466141747.0
2,Juice WRLD,hennessy,1.0,Robbery,330679687.0
3,Ariana Grande,tiffany,1.0,7 rings,270151652.0
4,Lizzo,dom,3.0,Truth Hurts,246878029.0
...,...,...,...,...,...
370,2 Chainz,birkin,1.0,Blue Cheese,1383082.0
371,Lil Yachty,henny,2.0,X Men,1321079.0
372,Drake,givenchy,1.0,All Me,1262995.0
373,21 Savage,birkin,1.0,Special,1181094.0


### Export final Pandas Dataframe to .csv

In [58]:
final_stream_df.to_csv('streams_lvmh_brands.csv')